## Проект - Телекоммуникационная компания
___

Заказчик исследования — телекоммуникационная компания, которая оказывает услуги на территории всего СНГ. 


**Цель проекта** - определить текущий уровень потребительской лояльности (NPS) среди клиентов из России. 

**План проекта**
- подключение к базе данных
- выгрузка данных
- создание Дашборда в Tableau

## Подключение к базе данных и выгрузка

In [1]:
import os
import pandas as pd
import numpy as np

from sqlalchemy import create_engine

In [2]:
path_to_db_local = 'telecomm_csi.db'
path_to_db_platform = '/datasets/telecomm_csi.db'
path_to_db = None

if os.path.exists(path_to_db_local):
    path_to_db = path_to_db_local
elif os.path.exists(path_to_db_platform):
    path_to_db = path_to_db_platform
else:
    raise Exception('Файл с базой данных SQLite не найден!')

if path_to_db:
    engine = create_engine(f'sqlite:///{path_to_db}', echo=False)

In [3]:
query = """
SELECT u.user_id,
       u.lt_day,
       CASE 
           WHEN u.lt_day <= 365 THEN 'new'
           ELSE 'old'
       END AS is_new,
       u.age,
       CASE 
           WHEN u.gender_segment = 1 THEN 'female'
           WHEN u.gender_segment = 0 THEN 'male'
           ELSE 'other'
       END AS gender_segment,
       u.os_name,
       u.cpe_type_name,
       loc.country,
       loc.city,
       SUBSTRING(a.title, 4) AS age_segment,
       SUBSTRING(t. title, 4) AS traffic_segment,
       SUBSTRING(lt.title, 4) AS lifetime_segment,
       u.nps_score,
       CASE 
           WHEN u.nps_score >= 9 THEN 'promoter'
           WHEN u.nps_score <= 6 THEN 'detractor'
           ELSE 'passive'
       END AS nps_group
FROM user AS u
LEFT JOIN location AS loc ON u.location_id = loc.location_id
LEFT JOIN age_segment AS a ON u.age_gr_id = a.age_gr_id
LEFT JOIN traffic_segment AS t ON u.tr_gr_id = t.tr_gr_id
LEFT JOIN lifetime_segment AS lt ON u.lt_gr_id = lt.lt_gr_id
""" 

Получили доступ к базе данных. Данные выгрузили в SQLite.

Собрал в одну витрину данные из разных таблиц. Эту витрину я буду использовать для дашборда.

In [4]:
df = pd.read_sql(query, engine)
df

,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_group
0,A001A2,2320,old,45.0,female,ANDROID,SMARTPHONE,Россия,Уфа,45-54,1-5,36+,10,promoter
1,A001WF,2344,old,53.0,male,ANDROID,SMARTPHONE,Россия,Киров,45-54,1-5,36+,10,promoter
2,A003Q7,467,old,57.0,male,ANDROID,SMARTPHONE,Россия,Москва,55-64,20-25,13-24,10,promoter
3,A004TB,4190,old,44.0,female,IOS,SMARTPHONE,Россия,РостовнаДону,35-44,0.1-1,36+,10,promoter
4,A004XT,1163,old,24.0,male,ANDROID,SMARTPHONE,Россия,Рязань,16-24,5-10,36+,10,promoter
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
502488,ZZZKLD,1249,old,54.0,female,ANDROID,SMARTPHONE,Россия,Москва,45-54,1-5,36+,5,detractor
502489,ZZZLWY,129,new,31.0,male,ANDROID,SMARTPHONE,Россия,Москва,25-34,1-5,4-6,8,passive
502490,ZZZQ5F,522,old,36.0,male,ANDROID,SMARTPHONE,Россия,Сургут,35-44,25-30,13-24,10,promoter
502491,ZZZQ8E,2936,old,37.0,female,ANDROID,SMARTPHONE,Россия,УланУдэ,35-44,65-70,36+,9,promoter


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 502493 entries, 0 to 502492
Data columns (total 14 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   user_id           502493 non-null  object 
 1   lt_day            502493 non-null  int64  
 2   is_new            502493 non-null  object 
 3   age               501939 non-null  float64
 4   gender_segment    502493 non-null  object 
 5   os_name           502493 non-null  object 
 6   cpe_type_name     502493 non-null  object 
 7   country           502493 non-null  object 
 8   city              502493 non-null  object 
 9   age_segment       502493 non-null  object 
 10  traffic_segment   502493 non-null  object 
 11  lifetime_segment  502493 non-null  object 
 12  nps_score         502493 non-null  int64  
 13  nps_group         502493 non-null  object 
dtypes: float64(1), int64(2), object(11)
memory usage: 53.7+ MB


In [6]:
df.to_csv('telecomm_csi_tableau.csv', index=False)

## Создание графиков, дашбордов и презентации в Tableau

https://public.tableau.com/views/TelecomProjectbySergeyToboev/TelecomProject?:language=en-US&:sid=&:redirect=auth&:display_count=n&:origin=viz_share_link